In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from tabulate import tabulate
import numpy as np
#Loading the cifar-10 dataset

 
train_dataset = dsets.CIFAR10(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)
 
test_dataset = dsets.CIFAR10(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

comparison_array = [['Pooling type','Accuracy']]
 


In [0]:
#Making Datasets Iterable with batch size of 100
 
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)
 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)
 


In [0]:
#Network Structure with Max Pooling in Pooling layer

'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
         
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        #Batch Normalization 
        self.batchnorm1=nn.BatchNorm2d(32)
                 
                      
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        #Batch Normalization 
        self.batchnorm2=nn.BatchNorm2d(32)
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        #Dropout 1
        self.fc1_drop = nn.Dropout(0.2)
        
        
        # Convolution 3
        self.cnn3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        #Batch Normalization 
        self.batchnorm3=nn.BatchNorm2d(64)
        
        # Convolution 4
        self.cnn4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        #Batch Normalization 
        self.batchnorm4=nn.BatchNorm2d(64)
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        #Dropout 2
        self.fc2_drop = nn.Dropout(0.2)
        
        # Convolution 5
        self.cnn5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu5 = nn.ReLU()
        #Batch Normalization 
        self.batchnorm5=nn.BatchNorm2d(128)
        
        # Convolution 6
        self.cnn6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu6 = nn.ReLU()
        #Batch Normalization 
        self.batchnorm6=nn.BatchNorm2d(128)
        #Max pool 3        
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        #Dropout 3
        self.fc3_drop = nn.Dropout(0.2)
               
        
         
        #Fully connected 1 (readout)
        self.fc1 = nn.Linear(128*4*4, 10) 
    
    def forward(self, x):
        # x =  torch.Size([100, 3, 32, 32])
        # Convolution 1
        out = self.cnn1(x) #torch.Size([100, 32, 32, 32])
        out = self.relu1(out) #torch.Size([100,32, 32, 32])
        out = self.batchnorm1(out)
        # Convolution 2
        out = self.cnn2(out) #torch.Size([100, 32, 32, 32])
        out = self.relu2(out) #torch.Size([100, 32, 32, 32])
        out=self.batchnorm2(out)
        # Max pool 1
        out = self.maxpool1(out) #torch.Size([100, 32, 16, 16])
        out = self.fc1_drop(out)
        
        # Convolution 3
        out = self.cnn3(out) #torch.Size([100, 64, 16, 16])
        out = self.relu3(out) #torch.Size([100, 64, 16, 16])
        out=self.batchnorm3(out)
        
        # Convolution 4
        out = self.cnn4(out) #torch.Size([100, 64, 16, 16])
        out = self.relu4(out) #torch.Size([100, 64, 16, 16])
        out=self.batchnorm4(out)
        # Max pool 2
        out = self.maxpool2(out) #torch.Size([100, 64, 8, 8])
        out = self.fc2_drop(out)
         
        
         # Convolution 5
        out = self.cnn5(out) #torch.Size([100, 128, 8, 8])
        out = self.relu5(out) #torch.Size([100, 128, 8, 8])
        out=self.batchnorm5(out)
        
         # Convolution 6
        out = self.cnn6(out) #torch.Size([100, 128, 8, 8])
        out = self.relu6(out) #torch.Size([100, 128, 8, 8])
        out=self.batchnorm6(out)
        # Max pool 3
        out = self.maxpool3(out) #torch.Size([100,128, 4, 4])
        out = self.fc3_drop(out)
         
         

        out = out.view(out.size(0), -1)   
 
        # Linear function (readout)
        out = self.fc1(out)
         
        return out
      
     

In [0]:
#Instantiate Model Class
 
model = CNNModel()
 
if torch.cuda.is_available():
    model.cuda()

In [0]:
#Instantiate Loss Class

criterion = nn.CrossEntropyLoss()

In [0]:

#Instantiate Optimizer Class
learning_rate = 0.01
 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
 

In [0]:
#Training the Model
iter = 0
arr=['MAX Pooling']
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
         
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

         
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
         
        iter += 1
         
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                
                if torch.cuda.is_available():
                    images = images.cuda()
                 
                # Forward pass only to get logits/output
                outputs = model(images)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum().float()

             
            accuracy = 100. * correct / total
             
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))
            
arr.append(round(float(accuracy),2))
comparison_array.append(arr) 

In [0]:
#function definition for Visualing the kernels
def filter_visualization(kernels,x,y):
  kernels = kernels - kernels.min()
  kernels = kernels / kernels.max()
  fig = plt.figure()
  plt.figure(figsize=(y,x))
  for idx, filt  in enumerate(kernels):
      plt.subplot(x,y, idx + 1)
      plt.imshow(filt[0, :, :])
      plt.axis('off')
    
    
  fig.show()

In [0]:
#Visualizing the 1st CNN layer kernels
print('Visualization of 1st CNN layer Kernels')
kernels = model.cnn1.weight.cpu().detach().clone()
filter_visualization(kernels,4,8)

In [0]:
#Visualizing the 2nd CNN layer kernels
print('Visualization of 2nd CNN layer Kernels')
kernels = model.cnn2.weight.cpu().detach().clone()
filter_visualization(kernels,4,8)

In [0]:
#Visualizing the 3rd CNN layer kernels
print('Visualization of 3rd CNN layer Kernels')
kernels = model.cnn3.weight.cpu().detach().clone()
filter_visualization(kernels,4,16)

In [0]:
#Visualizing the 4th CNN layer kernels
print('Visualization of 4th CNN layer Kernels')
kernels = model.cnn4.weight.cpu().detach().clone()
filter_visualization(kernels,4,16)

In [0]:
#Visualizing the 5th CNN layer kernels
print('Visualization of 5th CNN layer Kernels')
kernels = model.cnn5.weight.cpu().detach().clone()
filter_visualization(kernels,8,16)

In [0]:
#Visualizing the 6th CNN layer kernels
print('Visualization of 6th CNN layer Kernels')
kernels = model.cnn6.weight.cpu().detach().clone()
filter_visualization(kernels,8,16)

In [0]:
#function definition for visualizing activation map
def Activation_Map_Plot(tensor,r,c):
  tensor=tensor.squeeze()
  fig = plt.figure(figsize=(c,r))
  for i in range(r*c):
    ax1 = fig.add_subplot(r,c,i+1)
    ax1.imshow(tensor[i])
    ax1.axis('off')
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])
    
  plt.subplots_adjust(wspace=0.3, hspace=0.3)
  plt.show()

In [0]:
#Activation maps of different layers
from torch.autograd import Variable
for images, labels in test_loader:
  images = Variable(images.view(-1, 3*32*32))
  image = images[0].view(1, 3, 32, 32).cuda()
      
#1st CNN layer
out1 = model.cnn1(image)
print('Activation map of Conv layer 1')
Activation_Map_Plot(out1.data.cpu().numpy(),2,16)

#2nd CNN layer
out2 = model.cnn2(out1)
print('Activation map of Conv layer 2')
Activation_Map_Plot(out2.data.cpu().numpy(),2,16)

#3rd CNN layer
out3 = model.cnn3(out2)
print('Activation map of Conv layer 3')
Activation_Map_Plot(out3.data.cpu().numpy(),8,8)
  
#4th CNN layer
out4 = model.cnn4(out3)
print('Activation map of Conv layer 4')
Activation_Map_Plot(out4.data.cpu().numpy(),8,8)

#5th CNN layer
out5 = model.cnn5(out4)
print('Activation map of Conv layer 5')
Activation_Map_Plot(out5.data.cpu().numpy(),8,16)

#6th CNN layer
out6 = model.cnn6(out5)
print('Activation map of Conv layer 6')
Activation_Map_Plot(out6.data.cpu().numpy(),8,16)

In [0]:
#Network Structure with Average Pooling in Pooling layer
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
         
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        #Batch Normalization 1
        self.batchnorm1=nn.BatchNorm2d(32)
                 
            
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        #Batch Normalization 2
        self.batchnorm2=nn.BatchNorm2d(32)
        # Average pool 1
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        #Dropout 1
        self.fc1_drop = nn.Dropout(0.2)
        
        # Convolution 3
        self.cnn3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        #Batch Normalization 3
        self.batchnorm3=nn.BatchNorm2d(64)
        
        # Convolution 4
        self.cnn4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        #Batch Normalization 4
        self.batchnorm4=nn.BatchNorm2d(64)
        # Average pool 2
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        #Dropout 2
        self.fc2_drop = nn.Dropout(0.2)
        
        # Convolution 5
        self.cnn5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu5 = nn.ReLU()
        #Batch Normalization 5
        self.batchnorm5=nn.BatchNorm2d(128)
        
        # Convolution 6
        self.cnn6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu6 = nn.ReLU()
        #Batch Normalization 6
        self.batchnorm6=nn.BatchNorm2d(128)
        
        # Average pool 3
        self.avgpool3 = nn.AvgPool2d(kernel_size=2)
        #Dropout 3
        self.fc3_drop = nn.Dropout(0.2)
        
         
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(128*4*4, 10) 
    
    def forward(self, x):
        # x =  torch.Size([100, 3, 32, 32])
        # Convolution 1
        out = self.cnn1(x) #torch.Size([100, 32, 32, 32])
        
        out = self.relu1(out) #torch.Size([100, 32, 32, 32])
        out = self.batchnorm1(out)
        # Convolution 2
        out = self.cnn2(out) #torch.Size([100, 32, 32, 32])
        out = self.relu2(out) #torch.Size([100, 32, 32, 32])
        out=self.batchnorm2(out)
        # Average pool 1
        out = self.avgpool1(out) #torch.Size([100, 32, 16, 16])
        out = self.fc1_drop(out)
        
        # Convolution 3
        out = self.cnn3(out) #torch.Size([100, 64, 16, 16])
        out = self.relu3(out) #torch.Size([100, 64, 16, 16])
        out=self.batchnorm3(out)
        
        # Convolution 4
        out = self.cnn4(out) #torch.Size([100, 64, 16, 16])
        out = self.relu4(out) #torch.Size([100, 64, 16, 16])
        out=self.batchnorm4(out)
        # Average pool 2
        out = self.avgpool2(out) #torch.Size([100, 64, 8, 8])
        out = self.fc2_drop(out)
         
        
         # Convolution 5
        out = self.cnn5(out) #torch.Size([100, 128, 8, 8])
        out = self.relu5(out) #torch.Size([100, 128, 8, 8])
        out=self.batchnorm5(out)
        
         # Convolution 6
        out = self.cnn6(out) #torch.Size([100, 128, 8, 8])
        out = self.relu6(out) #torch.Size([100, 128, 8, 8])
        out=self.batchnorm6(out)
        # Average pool 3
        out = self.avgpool3(out) #torch.Size([100, 128, 4, 4])
        out = self.fc3_drop(out)
        
         

        out = out.view(out.size(0), -1)   
 
        # Linear function (readout)
        out = self.fc1(out)
        
    
        return out

In [0]:
#Training the Network with average pooling
iter = 0
arr=['AVG Pooling']
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
         
        
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

         
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
         
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                
                if torch.cuda.is_available():
                    images = images.cuda()
                 
                # Forward pass only to get logits/output
                outputs = model(images)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum().float()

             
            accuracy = 100. * correct / total
             
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))
arr.append(round(float(accuracy),2))
comparison_array.append(arr)

In [0]:
print(tabulate(comparison_array,
       headers="firstrow"))

In [0]:
if(comparison_array[1][1]>comparison_array[2][1]):
   print(str(comparison_array[1][0])+' has better accuracy over '+str(comparison_array[2][0]))
else:
  
   print(str(comparison_array[2][0])+' has better accuracy over '+str(comparison_array[1][0]))